In [ ]:
import logging
import ebooklib
from dotenv import load_dotenv
# from env_tokens import TELEGRAM_BOT_TOKEN, MISTRAL_API_ENDPOINT, PINECONE_API_KEY
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes
from mistralai import Mistral
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
from ebooklib import epub
from bs4 import BeautifulSoup
from langchain.text_splitter import CharacterTextSplitter
import os

load_dotenv()

TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
MISTRAL_API_ENDPOINT = os.getenv("MISTRAL_API_ENDPOINT")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)
logger = logging.getLogger(__name__)

# Токен Telegram бота
TELEGRAM_BOT_TOKEN = TELEGRAM_BOT_TOKEN

# API endpoint Mistral
MISTRAL_API_ENDPOINT = MISTRAL_API_ENDPOINT
MISTRAL_MODEL = 'mistral-medium'  # Replace with the name of your Mistral model
MISTRAL_CLIENT = Mistral(api_key=MISTRAL_API_ENDPOINT)

# API endpoint Pinecone
PINECONE_API_KEY = PINECONE_API_KEY
PINECONE_ENVIRONMENT = 'us-east-1'
INDEX_NAME = 'interview-qa2'

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Initialize SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
#model = SentenceTransformer('intfloat/multilingual-e5-large')

ind_text_dict = {}


async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text(
        'Hi! I am your Mistral RAG bot. Send me a message and I will retrieve and generate a response for you.')


async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    user_message = update.message.text
    logger.info(f"Received message: {user_message}")

    # Step 1: Retrieve relevant documents from the vector database
    retrieved_docs = retrieve_documents(user_message)

    # Step 2: Generate a response using Mistral API
    response = generate_response(user_message, retrieved_docs)

    await update.message.reply_text(response)


def retrieve_documents(query: str):
    # Convert the query to a vector
    query_vector = model.encode(query).tolist()

    # Query the Pinecone index with keyword arguments
    index = pc.Index(INDEX_NAME)
    results = index.query(vector=query_vector, top_k=5, include_metadata=True)
    
    # Log the retrieved results for debugging
    logger.info(f"Retrieved results: {results}")

    retrieved_docs = []
    for match in results['matches']:
        doc_id = match['id']
        try:
            retrieved_docs.append(ind_text_dict[doc_id])
        except KeyError:
            logger.warning(f"Document ID {doc_id} not found in local dictionary.")
    
    return retrieved_docs



def generate_response(query: str, documents: list):
    """Generate a response using Mistral API."""
    prompt = f"""
    You are an AI assistant designed to help users prepare for ML engineer interviews. 
    You have access to a knowledge base with information in English. 
    When a user asks a question, you should retrieve the relevant information from the knowledge base and then translate the response into the russian language.
    Knowledge base: {documents}
    
    Here is the user's question:
    [{query}]
    
    Please provide the answer only in the russian language.
        """

    # Send the prompt to Mistral API
    try:
        response = MISTRAL_CLIENT.chat.complete(
            model=MISTRAL_MODEL,
            messages=[{'role': "user", 'content': prompt}]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        logger.error(f"Error generating response: {e}")
        return "Sorry, I couldn't generate a response at the moment."


def extract_text_from_epub(file_path):
    book = epub.read_epub(file_path)
    text = []
    for item in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
        soup = BeautifulSoup(item.get_body_content(), 'html.parser')
        text.append(soup.get_text())
    return ' '.join(text)


def generate_embeddings(documents):
    embeddings = []
    for text_id, text in documents.items():
        embedding = model.encode(text)
        embeddings.append({'id': text_id, 'values': embedding.tolist()})
    return embeddings

2025-01-04 10:49:15,821 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/pinecone_plugins'])
2025-01-04 10:49:15,826 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference
2025-01-04 10:49:15,827 - pinecone_plugin_interface.logging - INFO - Installing plugin inference into Pinecone
2025-01-04 10:49:15,841 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2025-01-04 10:49:15,842 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: paraphrase-MiniLM-L6-v2


In [24]:
INDEX_NAME = 'interview-qa2'
epub_folder = '/home/bullat/projects/rag/Interview-2.0/data'
documents = []
for filename in os.listdir(epub_folder):
    if filename.endswith('.epub'):
        file_path = os.path.join(epub_folder, filename)
        text = extract_text_from_epub(file_path)
        #TODO использовать name для ссылки на источник
        documents.append({'name': filename, 'text': text})

text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
texts = []
for doc in documents:
    chunks = text_splitter.split_text(doc['text'])
    for chunk in chunks:
        text_id = f"{doc['name']}_{chunks.index(chunk)}" 
        ind_text_dict[text_id] = chunk
    # texts.extend(text_splitter.split_text(doc['text']))
# print(ind_text_dict[0])

embeddings = generate_embeddings(ind_text_dict)
# print(embeddings[0])
# Create the index if it doesn't exist
if INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=INDEX_NAME,
        dimension=len(embeddings[0]['values']),
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Upsert embeddings to the Pinecone index
index = pc.Index(INDEX_NAME)
index.upsert(embeddings)

/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):
2025-01-04 10:49:26,273 - langchain_text_splitters.base - WARNING - Created a chunk of size 3558, which is longer than the specified 2000
2025-01-04 10:49:26,275 - langchain_text_splitters.base - WARNING - Created a chunk of size 3734, which is longer than the specified 2000
2025-01-04 10:49:26,275 - langchain_text_splitters.base - WARNING - Created a 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-04 10:49:55,301 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/pinecone_plugins'])
2025-01-04 10:49:55,302 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference


{'upserted_count': 696}

In [25]:
import json

# Function to load questions from JSON file
def load_questions(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Function to save answers to JSON file
def save_answers(answers, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(answers, file, ensure_ascii=False, indent=4)

# Function to get answers from RAG
def get_answers(questions):
    answers = {}
    for q_id, question in questions.items():
        # Step 1: Retrieve relevant documents from the vector database
        retrieved_docs = retrieve_documents(question)

        # Step 2: Generate a response using Mistral API
        answer = generate_response(question, retrieved_docs)

        answers[q_id] = answer
    return answers

# Load questions from file
questions_file_path = '/home/bullat/projects/rag/Interview-2.0/question_for_test.json'
questions = load_questions(questions_file_path)

# Get answers from RAG
answers = get_answers(questions)

# Save answers to file
answers_file_path = '/home/bullat/projects/rag/Interview-2.0/answers.json'
save_answers(answers, answers_file_path)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-04 10:50:00,197 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/pinecone_plugins'])
2025-01-04 10:50:00,199 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference
2025-01-04 10:50:01,134 - __main__ - INFO - Retrieved results: {'matches': [{'id': 'John Mongan - Programming Interviews Exposed_ Coding Your '
                    'Way Through the Interview (2018, Wrox Press) - '
                    'libgen.li.epub_181',
              'score': 0.315634668,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds of '
                    'fully solved job interview questions from a wide range of '
                    'key topics in AI-Interviews AI (2020).epub_37',
              'score': 0.282967895,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds o

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-04 10:50:10,619 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/pinecone_plugins'])
2025-01-04 10:50:10,621 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference
2025-01-04 10:50:11,321 - __main__ - INFO - Retrieved results: {'matches': [{'id': 'John Mongan - Programming Interviews Exposed_ Coding Your '
                    'Way Through the Interview (2018, Wrox Press) - '
                    'libgen.li.epub_181',
              'score': 0.283457965,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds of '
                    'fully solved job interview questions from a wide range of '
                    'key topics in AI-Interviews AI (2020).epub_52',
              'score': 0.271393955,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds o

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

2025-01-04 11:15:28,883 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/pinecone_plugins'])
2025-01-04 11:15:28,885 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference
2025-01-04 11:15:29,595 - __main__ - INFO - Retrieved results: {'matches': [{'id': 'John Mongan - Programming Interviews Exposed_ Coding Your '
                    'Way Through the Interview (2018, Wrox Press) - '
                    'libgen.li.epub_181',
              'score': 0.326951474,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds of '
                    'fully solved job interview questions from a wide range of '
                    'key topics in AI-Interviews AI (2020).epub_37',
              'score': 0.279266417,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds o

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-04 11:15:39,593 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/pinecone_plugins'])
2025-01-04 11:15:39,594 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference
2025-01-04 11:15:40,285 - __main__ - INFO - Retrieved results: {'matches': [{'id': 'John Mongan - Programming Interviews Exposed_ Coding Your '
                    'Way Through the Interview (2018, Wrox Press) - '
                    'libgen.li.epub_181',
              'score': 0.296679169,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds of '
                    'fully solved job interview questions from a wide range of '
                    'key topics in AI-Interviews AI (2020).epub_37',
              'score': 0.265440583,
              'values': []},
             {'id': 'John Mongan - Programming Interviews Exposed_ Coding

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-04 11:15:46,050 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/pinecone_plugins'])
2025-01-04 11:15:46,052 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference
2025-01-04 11:15:46,731 - __main__ - INFO - Retrieved results: {'matches': [{'id': 'John Mongan - Programming Interviews Exposed_ Coding Your '
                    'Way Through the Interview (2018, Wrox Press) - '
                    'libgen.li.epub_181',
              'score': 0.291864961,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds of '
                    'fully solved job interview questions from a wide range of '
                    'key topics in AI-Interviews AI (2020).epub_37',
              'score': 0.280926913,
              'values': []},
             {'id': 'Shlomo Kashani - Deep Learning Interviews Hundreds o

In [26]:
import json
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Load your data
with open('/home/bullat/projects/rag/Interview-2.0/answers.json', 'r', encoding='utf-8') as f:
    generated_answers = json.load(f)

with open('/home/bullat/projects/rag/Interview-2.0/Answer_fact.json', 'r', encoding='utf-8') as f:
    reference_answers = json.load(f)

# Initialize the SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Define function to calculate Cosine Similarity
def calculate_cosine_similarity(generated, reference):
    generated_vector = model.encode([generated])
    reference_vector = model.encode([reference])
    return cosine_similarity(generated_vector, reference_vector)[0][0]

# Define function to calculate BLEU Score
def calculate_bleu_score(generated, reference):
    reference_tokens = [reference.split()]
    generated_tokens = generated.split()
    return sentence_bleu(reference_tokens, generated_tokens)

# Calculate and print metrics
for key in generated_answers.keys():
    generated = generated_answers[key]
    reference = reference_answers[key]
    
    cosine_sim = calculate_cosine_similarity(generated, reference)
    bleu_score = calculate_bleu_score(generated, reference)
    
    print(f"Question ID: {key}")
    print(f"Generated Answer: {generated}")
    print(f"Reference Answer: {reference}")
    print(f"Cosine Similarity: {cosine_sim}")
    print(f"BLEU Score: {bleu_score}")
    print()



2025-01-04 11:16:01,006 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2025-01-04 11:16:01,006 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: paraphrase-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question ID: 0
Generated Answer: Алгоритм бинарного поиска — это эффективный алгоритм поиска, который используется для нахождения элемента в отсортированном списке. Он основан на принципе деления пополам: в каждом шаге алгоритм проверяет средний элемент списка и определяет, находится ли искомое значение слева, справа или же оно является средним элементом. Затем алгоритм продолжает поиск в соответствующей половине списка, и так далее, пока элемент не будет найден или список не станет пустым.

Бинарный поиск имеет логарифмическую сложность O(log n), что делает его намного более эффективным, чем линейный поиск, особенно для больших списков.

Пример кода на Python:
```
def binary_search(arr, low, high, x):
  if high >= low:
    mid = (high + low) // 2
    if arr[mid] == x:
      return mid
    elif arr[mid] > x:
      return binary_search(arr, low, mid - 1, x)
    else:
      return binary_search(arr, mid + 1, high, x)
  else:
    return -1
```
В этом коде `arr` — отсортированный список, `

/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


...

Question ID: 98
Generated Answer: Ансамблевые методы в машинном обучении - это техники, которые комбинируют несколько моделей машинного обучения для улучшения точности прогнозов. Они основаны на идее, что ensemble ("ансамбль") из нескольких моделей может быть более точным, чем каждая модель отдельно. Существует несколько разных типов ансамблевых методов, таких как метод баггинга, метод бостонской шкалы и метод gradient boosting. Эти методы используются для решения задач классификации, регрессии и прогнозирования. Например, метод gradient boosting часто используется для решения задач регрессии и прогнозирования, поскольку он может эффективно моделировать сложные взаимосвязи между переменными. Ансамблевые методы широко используются в машинном обучении, поскольку они могут значительно улучшить точность моделей.
Reference Answer: Ансамблевые методы в машинном обучении объединяют несколько моделей для улучшения их точности и устойчивости. Основные методы включают bagging, boosting и stackin

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question ID: 99
Generated Answer: Когда вы оцениваете качество модели машинного обучения на тестовых данных, вы можете использовать различные метрики, такие как точность, recall, F1-мерa, log-потерю, ROC-карту и AUC-значение, зависящие от конкретной задачи. Вы можете также использовать кросс-валидацию для улучшения оценки качества модели. Кроме того, важно сравнивать результаты с базовыми моделями и моделями сопоставимого качества. Наконец, важно не надметать моделью, проверяя ее на данных, которые использовались для обучения модели. Это может привести к завышенной оценке качества модели.

Translation:

Когда вы оцениваете качество модели машинного обучения на тестовых данных, вы можете использовать различные метрики, такие как точность, полнота, F1-мерa, log-потеря, ROC-кривая и AUC-значение, в зависимости от конкретной задачи. Вы также можете использовать кросс-валидацию для улучшения оценки качества модели. Кроме того, важно сравнивать результаты с базовыми моделями и моделями сравн

/home/bullat/miniconda3/envs/rag/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [27]:
import json
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np
import nltk

# Download required NLTK data
nltk.download('punkt')

def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def simple_tokenize(text):
    # Simple whitespace tokenization with punctuation handling
    return [word.strip('.,!?()[]{}:;"\'') for word in text.split()]

def calculate_metrics(pred_answers, true_answers):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
    smooth = SmoothingFunction()
    metrics = {}
    
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    bleu_scores = []
    
    for key in pred_answers:
        if key in true_answers:
            rouge = scorer.score(true_answers[key], pred_answers[key])
            for metric in rouge_scores:
                rouge_scores[metric].append(rouge[metric].fmeasure)
            
            reference = [simple_tokenize(true_answers[key].lower())]
            candidate = simple_tokenize(pred_answers[key].lower())
            
            if reference[0] and candidate:  # Check if tokens exist
                bleu = sentence_bleu(reference, candidate, 
                                   smoothing_function=smooth.method1)
                bleu_scores.append(bleu)
    
    # Calculate averages
    metrics['rouge1'] = np.mean(rouge_scores['rouge1']) if rouge_scores['rouge1'] else 0
    metrics['rouge2'] = np.mean(rouge_scores['rouge2']) if rouge_scores['rouge2'] else 0
    metrics['rougeL'] = np.mean(rouge_scores['rougeL']) if rouge_scores['rougeL'] else 0
    metrics['bleu'] = np.mean(bleu_scores) if bleu_scores else 0
    
    return metrics

# Load and evaluate
pred_answers = load_json('/home/bullat/projects/rag/Interview-2.0/answers.json')
true_answers = load_json('/home/bullat/projects/rag/Interview-2.0/Answer_fact.json')
metrics = calculate_metrics(pred_answers, true_answers)

print("\nEvaluation Metrics:")
print(f"ROUGE-1: {metrics['rouge1']:.4f}")
print(f"ROUGE-2: {metrics['rouge2']:.4f}")
print(f"ROUGE-L: {metrics['rougeL']:.4f}")
print(f"BLEU: {metrics['bleu']:.4f}")

[nltk_data] Downloading package punkt to /home/bullat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-01-04 11:16:11,609 - absl - INFO - Using default tokenizer.



Evaluation Metrics:
ROUGE-1: 0.1352
ROUGE-2: 0.0675
ROUGE-L: 0.1337
BLEU: 0.0322


In [31]:
import json
import numpy as np
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from bert_score import score as bert_score

# Download required NLTK data
nltk.download('punkt')

# Initialize the SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # Replace with your chosen model
bert_scorer_model = 'roberta-large'  # Model to use for BERTScore

def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def simple_tokenize(text):
    # Simple whitespace tokenization with punctuation handling
    return [word.strip('.,!?()[]{}:;"\'') for word in text.split()]

def calculate_cosine_similarity(generated, reference):
    generated_vector = model.encode([generated])
    reference_vector = model.encode([reference])
    return cosine_similarity(generated_vector, reference_vector)[0][0]

def calculate_bert_score(pred_answers, true_answers):
    pred_list = [pred_answers[key] for key in pred_answers if key in true_answers]
    true_list = [true_answers[key] for key in pred_answers if key in true_answers]
    
    P, R, F1 = bert_score(pred_list, true_list, model_type=bert_scorer_model)
    return np.mean(F1.cpu().numpy())  # Convert tensors to numpy arrays

def calculate_metrics(pred_answers, true_answers):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
    smooth = SmoothingFunction()
    metrics = {}
    
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    bleu_scores = []
    cosine_scores = []
    
    for key in pred_answers:
        if key in true_answers:
            rouge = scorer.score(true_answers[key], pred_answers[key])
            for metric in rouge_scores:
                rouge_scores[metric].append(rouge[metric].fmeasure)
            
            reference = [simple_tokenize(true_answers[key].lower())]
            candidate = simple_tokenize(pred_answers[key].lower())
            
            if reference[0] and candidate:  # Check if tokens exist
                bleu = sentence_bleu(reference, candidate, 
                                   smoothing_function=smooth.method1)
                bleu_scores.append(bleu)
            
            # Calculate Cosine Similarity
            cosine_sim = calculate_cosine_similarity(pred_answers[key], true_answers[key])
            cosine_scores.append(cosine_sim)
    
    # Calculate BERTScore
    bert_f1_score = calculate_bert_score(pred_answers, true_answers)
    
    # Calculate averages
    metrics['rouge1'] = np.mean(rouge_scores['rouge1']) if rouge_scores['rouge1'] else 0
    metrics['rouge2'] = np.mean(rouge_scores['rouge2']) if rouge_scores['rouge2'] else 0
    metrics['rougeL'] = np.mean(rouge_scores['rougeL']) if rouge_scores['rougeL'] else 0
    metrics['bleu'] = np.mean(bleu_scores) if bleu_scores else 0
    metrics['cosine'] = np.mean(cosine_scores) if cosine_scores else 0
    metrics['bert_f1'] = bert_f1_score
    
    return metrics

# Load and evaluate
pred_answers = load_json('/home/bullat/projects/rag/Interview-2.0/answers.json')
true_answers = load_json('/home/bullat/projects/rag/Interview-2.0/Answer_fact.json')
metrics = calculate_metrics(pred_answers, true_answers)

print("\nEvaluation Metrics:")
print(f"ROUGE-1: {metrics['rouge1']:.4f}")
print(f"ROUGE-2: {metrics['rouge2']:.4f}")
print(f"ROUGE-L: {metrics['rougeL']:.4f}")
print(f"BLEU: {metrics['bleu']:.4f}")
print(f"Cosine Similarity: {metrics['cosine']:.4f}")
print(f"BERTScore F1: {metrics['bert_f1']:.4f}")


[nltk_data] Downloading package punkt to /home/bullat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-01-04 11:40:37,655 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2025-01-04 11:40:37,656 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: paraphrase-MiniLM-L6-v2
2025-01-04 11:40:40,860 - absl - INFO - Using default tokenizer.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Evaluation Metrics:
ROUGE-1: 0.1352
ROUGE-2: 0.0675
ROUGE-L: 0.1337
BLEU: 0.0322
Cosine Similarity: 0.8786
BERTScore F1: 0.9142


Метрики ROUGE и BLEU больше ориентируются на схожесть слов и выражений. RAG выдает ответы отличными от эталона словами, однако Cosine Similarity и BERTScore показывают, что у ответов схожая семантика и смысл.